# MNIST-CNN
This notebook shows how to perform handwritten digit classification using the CNN implemented with spooNN.

## 1. Preparation
First we import all the necessary components:

In [1]:
import sys
import math
import numpy as np
import os
import time
from PIL import Image
from matplotlib import pyplot
import cv2
from datetime import datetime
from loader import loader
import scipy.misc
from IPython.display import display

Next, we get the overlay that is in the same directory as this notebook. This is loaded onto the FPGA fabric on the PYNQ.
Then, a handle to the spooNN IP is obtained via nn_ctrl. Using this, we are able to write to the registers of the IP to configure it.

Got nn_ctrl!


Here, we set some parameters. MINIBATCH_SIZE is how many images we want to classify in one go. The rest of the parameters are related to image properties (28 by 28 images, 8 bits per pixel, and 448 bits per line resulting in 14 total lines to transmit one image to the FPGA).

In [2]:
## Parameters
MINIBATCH_SIZE = 10
height = 28
width = 28
pixel_bits = 8
pixels_per_line = 448/pixel_bits
num_lines = int((height*width)/pixels_per_line)
print('num_lines per image: ' + str(num_lines))

num_lines per image: 14


Let's download the data set if it is not yet there, by executing the following bash script. If you don't have internet access from your PYNQ, you might have to execute this script on your host machine and then scp mnist.t to this directory.

In [3]:
sex = ["3 149:29 150:135 151:189 152:233 153:238 154:238 155:220 156:162 157:73 176:43 177:232 178:255 179:255 180:212 181:201 182:201 183:229 184:255 185:255 186:172 187:15 204:149 205:255 206:110 207:23 211:2 212:51 213:181 214:255 215:112 232:85 233:184 234:2 241:39 242:255 243:154 269:109 270:255 271:104 296:23 297:237 298:241 299:22 324:180 325:255 326:100 351:122 352:255 353:178 378:48 379:255 380:210 406:37 407:255 408:241 409:109 410:12 435:79 436:228 437:255 438:230 439:96 464:14 465:116 466:239 467:255 468:187 469:60 494:24 495:166 496:255 497:255 498:83 524:59 525:248 526:221 552:7 553:244 554:214 580:183 581:255 582:96 606:20 607:157 608:255 609:168 631:6 632:43 633:134 634:243 635:255 636:182 637:13 649:74 650:141 651:143 652:142 653:142 654:142 655:141 656:165 657:187 658:213 659:236 660:255 661:255 662:224 663:108 677:172 678:255 679:255 680:255 681:255 682:255 683:255 684:255 685:252 686:234 687:210 688:170 689:92 690:8",
"4 190:24 191:5 210:21 211:154 212:46 217:20 218:239 219:117 237:85 238:226 239:255 240:61 245:56 246:255 247:170 263:30 264:189 265:255 266:237 267:89 273:61 274:255 275:171 290:101 291:248 292:255 293:129 294:17 301:58 302:255 303:191 316:9 317:152 318:255 319:212 320:65 329:66 330:255 331:207 343:57 344:219 345:255 346:147 347:7 357:112 358:255 359:204 370:111 371:255 372:247 373:93 385:139 386:255 387:225 396:41 397:196 398:255 399:178 400:28 413:184 414:255 415:241 424:222 425:255 426:228 427:88 428:26 429:7 440:33 441:244 442:255 443:255 452:118 453:215 454:253 455:255 456:255 457:243 458:221 459:194 460:166 461:167 462:148 463:122 464:122 465:128 466:179 467:212 468:253 469:255 470:240 471:255 481:1 482:38 483:77 484:146 485:177 486:206 487:232 488:250 489:248 490:255 491:255 492:255 493:255 494:247 495:213 496:178 497:114 498:142 499:255 515:1 516:11 517:10 518:26 519:37 520:35 521:35 522:12 526:115 527:255 554:121 555:255 582:121 583:255 610:121 611:255 638:121 639:255 666:124 667:255 694:115 695:255 722:18 723:64",
"5 177:37 178:50 179:60 180:86 181:115 182:127 183:124 184:139 185:167 186:193 187:209 188:234 189:237 190:53 204:47 205:250 206:255 207:255 208:255 209:255 210:255 211:255 212:255 213:249 214:231 215:212 216:192 217:160 218:31 232:69 233:255 234:156 235:96 236:68 237:47 238:32 239:33 240:29 241:10 242:1 260:67 261:255 262:73 288:65 289:255 290:95 316:39 317:255 318:127 344:33 345:255 346:129 372:34 373:255 374:124 400:36 401:255 402:141 403:43 404:75 405:107 406:134 407:132 408:142 409:174 410:176 411:176 412:176 413:174 414:135 415:4 428:27 429:255 430:255 431:255 432:255 433:255 434:255 435:255 436:255 437:243 438:239 439:239 440:239 441:252 442:255 443:49 457:82 458:136 459:117 460:83 461:53 462:28 463:27 464:26 465:7 466:3 467:4 468:3 469:87 470:255 471:73 497:77 498:255 499:72 525:84 526:255 527:71 553:150 554:255 555:37 580:5 581:239 582:204 608:88 609:255 610:103 635:15 636:226 637:236 638:10 654:6 662:79 663:222 664:255 665:93 681:34 682:230 683:153 684:112 685:113 686:113 687:111 688:136 689:213 690:255 691:235 692:104 709:30 710:225 711:255 712:255 713:255 714:255 715:255 716:255 717:222 718:120 719:15",
"6 100:41 101:200 102:58 127:2 128:188 129:255 130:89 155:156 156:255 157:182 158:9 182:100 183:255 184:199 185:6 209:10 210:231 211:247 212:34 237:112 238:255 239:132 264:9 265:229 266:245 267:19 292:91 293:255 294:146 320:173 321:253 322:32 347:15 348:235 349:251 350:174 351:172 352:118 353:35 375:27 376:250 377:255 378:255 379:255 380:255 381:255 382:138 403:86 404:255 405:209 406:45 407:35 408:112 409:229 410:255 411:168 412:3 431:193 432:255 433:108 437:20 438:179 439:255 440:113 458:25 459:248 460:255 461:86 466:25 467:252 468:211 486:31 487:255 488:255 489:58 495:199 496:253 497:25 514:4 515:190 516:255 517:55 523:172 524:255 525:39 543:142 544:255 545:46 550:21 551:236 552:240 553:12 571:134 572:255 573:110 577:8 578:189 579:255 580:123 599:39 600:239 601:255 602:182 603:151 604:165 605:223 606:255 607:159 628:50 629:202 630:255 631:255 632:255 633:245 634:152",
"7 231:77 232:227 233:175 234:149 235:106 236:78 237:58 238:42 239:42 240:42 241:42 242:43 243:38 259:52 260:208 261:255 262:255 263:255 264:255 265:255 266:255 267:255 268:255 269:255 270:255 271:252 272:99 289:21 290:38 291:73 292:92 293:121 294:132 295:130 296:130 297:130 298:153 299:255 300:138 326:25 327:255 328:137 354:84 355:255 356:103 382:146 383:255 384:46 409:4 410:223 411:233 412:3 437:63 438:255 439:142 465:153 466:255 467:49 492:3 493:231 494:225 520:61 521:255 522:138 548:142 549:255 550:51 575:1 576:228 577:228 578:4 603:54 604:255 605:149 631:139 632:255 633:56 658:7 659:226 660:229 661:4 686:67 687:255 688:126 714:136 715:255 716:55 741:3 742:221 743:248 744:10 769:9 770:229 771:156",
"2 151:31 152:139 153:215 154:238 155:238 156:223 157:176 158:92 159:14 178:94 179:247 180:255 181:236 182:201 183:201 184:226 185:255 186:255 187:236 188:113 205:60 206:255 207:236 208:92 209:8 212:1 213:35 214:128 215:231 216:255 217:104 232:5 233:210 234:247 235:49 243:47 244:255 245:170 260:18 261:255 262:188 271:8 272:245 273:200 288:18 289:255 290:184 299:14 300:248 301:194 316:7 317:237 318:247 319:19 327:53 328:255 329:158 345:101 346:205 347:24 355:152 356:255 357:74 382:16 383:242 384:222 410:93 411:255 412:120 437:2 438:209 439:255 440:29 465:152 466:255 467:133 492:103 493:255 494:196 519:99 520:255 521:217 522:25 546:70 547:255 548:226 549:35 573:81 574:252 575:244 576:43 599:3 600:158 601:255 602:233 603:58 626:57 627:203 628:255 629:206 630:59 654:197 655:255 656:255 657:253 658:247 659:232 660:225 661:202 662:185 663:165 664:141 665:142 666:142 667:143 668:100 669:1 682:37 683:103 684:155 685:171 686:179 687:213 688:216 689:242 690:251 691:255 692:255 693:255 694:255 695:255 696:222 697:5",
"1 155:130 156:164 183:192 184:236 211:182 212:224 239:182 240:224 267:182 268:224 295:182 296:224 323:182 324:224 351:182 352:224 379:182 380:224 407:182 408:224 435:182 436:224 463:182 464:224 491:182 492:224 519:182 520:224 547:182 548:224 575:182 576:224 603:182 604:224 631:182 632:224 659:189 660:232 687:155 688:195",
"3 151:16 152:161 153:237 154:244 155:244 156:229 157:155 158:44 178:4 179:203 180:255 181:191 182:169 183:170 184:204 185:255 186:242 187:25 206:5 207:204 208:124 213:102 214:255 215:85 241:50 242:255 243:99 269:116 270:255 271:67 296:16 297:234 298:220 299:7 323:4 324:175 325:255 326:78 350:52 351:204 352:255 353:149 375:9 376:48 377:142 378:252 379:239 380:103 402:133 403:245 404:255 405:255 406:233 407:74 430:91 431:180 432:220 433:249 434:255 435:255 436:229 437:160 438:79 439:30 460:3 461:35 462:77 463:137 464:198 465:255 466:255 467:255 468:205 469:110 493:31 494:85 495:143 496:250 497:246 498:4 523:12 524:239 525:204 551:190 552:255 553:57 578:129 579:255 580:118 604:10 605:161 606:255 607:179 631:73 632:220 633:255 634:145 655:77 656:113 657:116 658:172 659:255 660:238 661:93 682:3 683:216 684:255 685:255 686:255 687:161 688:22",
"5 149:79 150:220 151:217 152:199 153:196 154:164 155:129 156:82 157:53 158:36 159:37 160:29 177:145 178:255 179:241 180:240 181:248 182:255 183:255 184:255 185:255 186:255 187:255 188:255 189:72 205:112 206:255 207:85 208:6 209:15 210:35 211:65 212:105 213:140 214:155 215:155 216:142 217:29 233:67 234:255 235:138 261:15 262:245 263:218 264:2 290:184 291:255 292:28 318:118 319:255 320:80 346:51 347:255 348:172 375:218 376:241 403:154 404:255 405:116 406:84 407:85 408:85 409:85 410:85 411:86 412:65 413:5 431:108 432:255 433:255 434:255 435:255 436:255 437:255 438:255 439:255 440:255 441:166 459:25 460:188 461:130 462:92 463:93 464:93 465:93 466:92 467:93 468:210 469:253 470:31 496:102 497:255 498:105 524:54 525:255 526:144 552:44 553:255 554:145 580:39 581:255 582:145 608:70 609:255 610:147 633:14 634:52 635:117 636:238 637:255 638:69 654:2 655:105 656:145 657:176 658:185 659:201 660:225 661:246 662:255 663:255 664:239 665:97 682:10 683:231 684:255 685:255 686:251 687:243 688:218 689:199 690:149 691:100 692:19",
"1 153:96 154:196 155:45 181:161 182:255 183:154 209:75 210:255 211:220 212:3 237:14 238:234 239:255 240:69 266:153 267:255 268:150 294:67 295:255 296:226 297:2 322:20 323:243 324:251 325:46 351:191 352:255 353:90 379:153 380:255 381:136 407:98 408:255 409:182 435:73 436:255 437:206 463:37 464:252 465:241 466:19 491:11 492:233 493:255 494:39 519:7 520:226 521:255 522:50 548:196 549:255 550:81 576:171 577:255 578:116 604:112 605:255 606:172 632:87 633:255 634:196 660:59 661:255 662:232 663:4 688:18 689:231 690:184",
"2 122:67 123:176 124:225 125:239 126:195 127:69 149:103 150:255 151:255 152:222 153:201 154:255 155:255 156:167 157:30 176:36 177:255 178:229 179:55 182:49 183:181 184:255 185:229 186:35 204:85 205:255 206:106 212:114 213:255 214:144 232:60 233:255 234:138 240:7 241:247 242:201 260:8 261:238 262:230 263:14 268:3 269:235 270:207 289:123 290:255 291:83 296:5 297:236 298:205 317:10 318:110 319:22 324:5 325:236 326:205 352:3 353:236 354:208 380:21 381:251 382:192 408:97 409:255 410:117 436:208 437:255 438:29 463:107 464:255 465:149 490:39 491:245 492:238 493:19 517:12 518:211 519:255 520:88 544:11 545:195 546:255 547:128 571:46 572:204 573:255 574:128 598:108 599:248 600:255 601:155 626:194 627:255 628:255 629:229 630:198 631:180 632:150 633:137 634:138 635:138 636:138 637:138 638:138 639:138 640:138 641:139 642:132 643:33 654:19 655:124 656:168 657:203 658:242 659:255 660:255 661:255 662:255 663:255 664:255 665:255 666:255 667:255 668:255 669:255 670:255 671:90",
"1 127:153 128:158 155:231 156:237 183:219 184:225 211:219 212:225 239:219 240:225 267:219 268:225 295:219 296:225 323:219 324:225 351:219 352:225 379:219 380:225 407:219 408:225 435:219 436:225 463:219 464:225 491:219 492:225 519:219 520:225 547:220 548:223 575:216 576:232 577:4 603:182 604:255 605:23 631:182 632:255 633:25 659:146 660:227 661:10"
]

Next, we can load the images we want to classify to the memory and display them.

In [4]:
# Read images from file
l = loader()
l.load_libsvm_data_array(sex, num_features=784, one_hot=0, classes=None)
images = np.zeros((MINIBATCH_SIZE,28,28))
for i in range(0, MINIBATCH_SIZE):
    images[i,:,:] = (l.a[i].reshape((28,28))).astype('int')
    
# Display images
for i in range(0,MINIBATCH_SIZE):
    display(scipy.misc.toimage( images[i,:,:] ))

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  # Remove the CWD from sys.path while we load stuff.


## 2. Execution
Now we are ready to invoke the FPGA-based CNN to perform classification on the images that are in the memory.

- We first allocate the buffers that will be used by the DMA engine, to transfer images to the FPGA and get the results back.
- We write the images to the input buffer.
- We set a register (numReps) on the mnist-cnn IP, by writing MINIBATCH_SIZE to address 0x10. This is how many images the IP expects the DMA engine to send.
- We start the transfer on both receive and send channels.
- After the execution is complete, we display the classification results. Notice that they match the images we displayed earlier. 

In [6]:
in_buffer = xlnk.cma_array(shape=(MINIBATCH_SIZE*num_lines, 64), dtype=np.uint8)
out_buffer = xlnk.cma_array(shape=(MINIBATCH_SIZE, 16), dtype=np.int32)
print('allocated buffers')

for i in range(0,MINIBATCH_SIZE):
    in_buffer[i*num_lines:(i+1)*num_lines, 0:56] = np.reshape(images[i,:,:], (num_lines, 56))

start = time.time()
nn_ctrl.write(0x0, 0) # Reset
nn_ctrl.write(0x10, MINIBATCH_SIZE)
nn_ctrl.write(0x0, 1) # Deassert reset
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.transfer(in_buffer)
end = time.time()

time_per_image = (end-start)/MINIBATCH_SIZE
print("Time per image: " + str(time_per_image) + " s")
print("Images per second: " + str(1.0/time_per_image))

time.sleep(1)

for i in range(0,MINIBATCH_SIZE):
    print(str(np.argmax(out_buffer[i,:])))

allocated buffers
Time per image: 0.00024547576904296873 s
Images per second: 4073.721833721834
7
2
1
0
4
1
4
9
5
9


Since we worked only on 10 images in this notebook, the maximum image processing throughput capability of the CNN is not displayed. Try increasing the MINIBATCH_SIZE to observe a much higher throughput (You might want to comment out display part to skip displaying many images). 